# 08 - Ejercicio 1 EDA - Datos del Naufragio del Titanic


* Para este ejercicio disponemos de una serie de datos relativos a los ***pasajeros que se embarcaron en el Titanic***, así como un flag que nos indica si ***sobrevivieron o no***.


* Estos datos se encuentran en los siguientes ficheros:

    - ***Pasajeros***: ./data/titanic/pasajeros.csv
    - ***Supervivientes***: ./data/titanic/supervivientes.csv


* Los ***campos que hay en el fichero de Pasajeros*** son los siguientes:
    - PassengerId: identificador de pasajero
    - Pclass: clase en la que viajaba 1 = 1st, 2 = 2nd, 3 = 3rd
    - Name
    - Sex
    - Age
    - SibSp: hermanos o conyujes abordo
    - Parch: padres o hijos abordo
    - Ticket: id del ticket
    - Fare
    - Cabin: id de la cabina
    - Embarked: Puerto de embarcación C = Cherbourg, Q = Queenstown, S = Southampton

* Los ***campos que hay en el fichero de Supervivientes*** son los siguientes:
    - PassengerId:  identificador de pasajero
    - Survived: 1: sobrevivio, 0: murió

#### Para este ejercicio se pide:

* Dado un DataFrame (df) ya cargado con los datos de los pasajeros y si sobrevivieron o no:

    1. ***Crear una nueva columna en el DataFrame (df) llamada puerto, que contenga en Nombre del puerto*** (No el identificador del puerto)
    2. ***Obtener el número de pasajeros que embarcaron en cada puerto*** (Cherbourg, Queenstown, Southampton)
    3. ***Obtener cuantos hombres y mujeres embarcaron***
    4. ***Obtener la edad media de los hombres y mujeres que sobrevivieron y murieron***
    5. ***Obtener cuantos muertos hubo por rango de edad*** (< 18 años: jovenes, 18-65 años: adultos, 65 >: Anciano) 
    6. ***Obtener el número de muertos por clase y genero***
    7. ***Obtener el número de muertos y supervivientes por ciudad de origen***

<hr>

#### Pasos previos:

   - Importamos la librería de pandas
   - Cargamos los ficheros y los pasamos a un DataFrame
   - Creamos el DataFrame para realizar los ejercicios, realizando un ***inner join*** por el campo *'PassengerId'*

In [1]:
import pandas as pd

# Cargamos los ficheros
df_pasajeros = pd.read_csv('../data/titanic/pasajeros.csv', header=0)
df_supervivientes = pd.read_csv('../data/titanic/supervivientes.csv', header=0)

# Join
df = pd.merge(df_pasajeros, df_supervivientes, on='PassengerId', how='inner')

df.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
1196,1197,1,"Crosby, Mrs. Edward Gifford (Catherine Elizabe...",female,64.0,1,1,112901,26.55,B26,S,1.0
402,403,3,"Jussila, Miss. Mari Aina",female,21.0,1,0,4137,9.825,NaN,S,0.0
909,910,3,"Ilmakangas, Miss. Ida Livija",female,27.0,1,0,STON/O2. 3101270,7.925,NaN,S,1.0
451,452,3,"Hagland, Mr. Ingvald Olai Olsen",male,NaN,1,0,65303,19.9667,NaN,S,0.0
973,974,1,"Case, Mr. Howard Brown",male,49.0,0,0,19924,26,NaN,S,0.0


<hr>


#### 1. ***Crear una nueva columna en el DataFrame (df) llamada puerto, que contenga en Nombre del puerto*** (No el identificador del puerto)

In [2]:
df['puerto'] = df['Embarked'].apply(lambda x: 'Cherbourg' if x== 'C' else ('Southampton' if x=='S' else 'Queenstown'))
df.sample(5)

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived,puerto
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13,NaN,S,0.0,Southampton
1221,1222,2,"Davies, Mrs. John Morgan (Elizabeth Agnes Mary...",female,48.0,0,2,C.A. 33112,36.75,NaN,S,1.0,Southampton
69,70,3,"Kink, Mr. Vincenz",male,26.0,2,0,315151,8.6625,NaN,S,0.0,Southampton
619,620,2,"Gavey, Mr. Lawrence",male,26.0,0,0,31028,10.5,NaN,S,0.0,Southampton
1160,1161,3,"Pokrnic, Mr. Mate",male,17.0,0,0,315095,8.6625,NaN,S,0.0,Southampton


<hr>


#### 2. ***Obtener el número de pasajeros que embarcaron en cada puerto*** (Cherbourg, Queenstown, Southampton)

In [3]:
df.groupby(['puerto'])['puerto'].agg(['count']).reset_index()

,puerto,count
0,Cherbourg,269
1,Queenstown,126
2,Southampton,914


<hr>


#### 3. ***Obtener cuantos hombres y mujeres embarcaron***

In [4]:
df.groupby(['Sex'])['Sex'].agg(['count']).reset_index()

,Sex,count
0,female,466
1,male,843


<hr>


#### 4. Obtener la edad media de los hombres y mujeres que sobrevivieron y murieron

In [5]:
df.groupby(['Sex', 'Survived'])['Age'].agg(['mean']).reset_index()

,Sex,Survived,mean
0,female,0.0,25.046875
1,female,1.0,29.406142
2,male,0.0,31.129929
3,male,1.0,27.276022


<hr>


#### 5. ***Obtener cuantos muertos hubo por rango de edad*** (< 18 años: jovenes, 18-65 años: adultos, 65 >: Anciano)


* En primer lugar creamos una ***nueva columna*** llamada ***'rango_edad'***.


* Agregamos por la columna ***'rango_edad'*** y realizamos un ***conteo***.

In [6]:
# Nueva Columna
df['rango_edad'] = df['Age'].apply(lambda x: 'Joven' if x < 18 else ('Anciano' if x > 65 else 'Adulto'))

# Agregacion
df.groupby(['rango_edad'])['rango_edad'].agg(['count'])

,count
rango_edad,
Adulto,1145
Anciano,10
Joven,154


<hr>


#### 6. ***Obtener el número de muertos por clase y genero***


* En primer lugar tenemos que ***filtrar*** todos aquellos pasajeros que murieron.


* En segundo lugar ***agregamos y realizamos el conteo***.

In [7]:
df[df['Survived'] == 1].groupby(['Pclass', 'Sex'])['Survived'].agg(['count']).reset_index()

,Pclass,Sex,count
0,1,female,141
1,1,male,45
2,2,female,100
3,2,male,17
4,3,female,144
5,3,male,47


<hr>


#### 7. ***Obtener el número de muertos y supervivientes por ciudad de origen***

In [8]:
df.groupby(['Survived', 'puerto'])['Survived'].agg(['count']).reset_index()

,Survived,puerto,count
0,0.0,Cherbourg,137
1,0.0,Queenstown,69
2,0.0,Southampton,609
3,1.0,Cherbourg,132
4,1.0,Queenstown,57
5,1.0,Southampton,305


<hr>

*Este Notebook ha sido desarrollado por **Ricardo Moya García** y registrado en Safe Creative como ***Atribución-NoComercial-CompartirIgual***.*

<img src="../imgs/CC_BY-NC-SA.png" alt="CC BY-NC">